In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  4 14:56:42 2016
@author: Artem
"""
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
#Библиотека содержит
import get_data_dic
import distr
from tqdm import tqdm
#Вывод на экран текущего времени (час, мин)
localtime = time.localtime(time.time())
print ("Local current time :", localtime[3:5])
#Ссылкы на первую страницу выдачи при поиске
#квартир из определенного района при поиске квартир из ЦАО

mln_boarders=[]
for mln_price in range(1,50,1):
    mln_boarders.append(mln_price*1000000)
    
dist_list =[]
dist_list_show=[]
min_boarder=0
for boarder in mln_boarders:
    a = 'https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2'
    b = '&maxprice=' + str(boarder)
    c = '&minprice=' + str(min_boarder+1)
    d = '&offer_type=flat&'
    e = 'p={}'
    f = '&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
    dist_list_show.append(' from ' + str(min_boarder/1000000) + ' to ' + str(boarder/1000000)+' mln')
    min_boarder = boarder
    abcdef = a + b + c + d + e + f
    #print(abcdef)
    dist_list.append(abcdef)
#print(dist_list)
#dist_list=[cent]
#X - будущий большой датасет со всеми квартирами и фичами
N=30 #число страниц, по которым мы бегаем, ища
X=[]#all data
#Цикл по всем районам. Номер района в цикле это и есть номер,
#под которым этот районом будет закодирован в конечном датасете
ID=0 #номер квартиры в датасете
for distN in tqdm(range( len(dist_list) )):
    #print(dist_list_show[distN])
    time1 = time.localtime(time.time())
    #print ("Local current time :", time1[3:5])
    #по главной ссылке находим номера страниц каждой из квартир
    #например, на нулевой итерации links - список уникальных номеров всех
    #квартир из ЦАО
    links=distr.links( dist_list[distN],N )
    #пробегаем во всем квартирам в округе
    for link in tqdm(links):
        #print(link)        
        #словарь из фич для данной квартиры
        ind_data_dic=get_data_dic.get_data_dic (link)
        #добавляем поле District, которое зависит от района
        ind_data_dic['Search_group']=dist_list_show[distN]
        ind_data_dic['ID']=ID
        ind_data_dic['Link']='http://www.cian.ru/sale/flat/' + str(link) + '/'
        ID+=1
        #добавляем словарь фич для квартиры в итоговый датасет    
        X.append(ind_data_dic)
    #print("preparing data")
    data=pd.DataFrame(X)
    
    #print (time.strftime("%H_%M_%S"))
    #print (time.strftime("%d_%m_%Y"))
    data.to_csv("output\data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") + ".csv",sep=';')
data.to_csv("output\FINAL_data" +time.strftime("%d_%m_%Y")+'__'+ time.strftime("%H_%M_%S") + ".csv",sep=';')
print('Done')

111.76693048363846
Local current time : (22, 42)


  2%|█▋                                                                                 | 1/49 [00:20<16:09, 20.20s/it]

In [6]:
#data[['Balcony', 'Built_year', 'Description', 'Floor', 'Home_type', 'ID']]
#data[['Kitsp', 'Lift', 'Link', 'Livesp', 'Metro_station', 'Metro_time']]
#data[['Nfloor', 'Parking', 'Price', 'Remont', 'Roof_hight', 'Sales_type']]
#data[['Sdacha_GK', 'Search', 'Tel', 'Totsp', 'View', 'WC', 'Walk_metro']]
data[['dist', 'is_premium', 'lat', 'lon', 'publication_date', 'rooms','sale']]

,dist,is_premium,lat,lon,publication_date,rooms,sale
0,4.748564,"false,",55.724636,37.581748,1483786642,3,"""sale"","
1,6.945398,"false,",55.859261,37.602058,1483812362,1,"""sale"","


In [1]:
def heat_map(x,y):
    import numpy as np
    import matplotlib.pyplot as plt
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=200)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.clf()
    #plt.figure(figsize=(10,5))
    plt.imshow(heatmap.T, extent=extent, origin='lower')
    plt.show()
